In [ ]:
import pandas as pd
import numpy as np
# making data frame  
input = open("input.txt", "r")
Lines = input.readlines()
grid_arr = []
rules = []

In [ ]:
# Strips the newline character
for i in range(0 ,2):
   rules.append(Lines[i].strip())
rules

['greedyBestFirst', '4']

In [ ]:
for i in range(2 ,len(Lines)):
   grid_arr.append(Lines[i].strip())
grid_arr

['ADRB', 'RRRW', 'DRDR', 'DRRR']

In [ ]:
sentence = ""
for word in grid_arr:
    sentence += str(word)
number_columns = int(rules[1])
l = [list(sentence[i:i+number_columns]) for i in range(0, len(sentence), number_columns)]
matrix = [s if len(s) == number_columns else s+[None]*(number_columns-len(s)) for s in l]
matrix

[['A', 'D', 'R', 'B'],
 ['R', 'R', 'R', 'W'],
 ['D', 'R', 'D', 'R'],
 ['D', 'R', 'R', 'R']]

In [ ]:
for i in range(len(matrix)):
  for j in range(len(matrix)):
    if matrix[i][j] == "R":
      matrix[i][j] = 1
    if matrix[i][j] == "D":
      matrix[i][j] = 4
    if matrix[i][j] == "B":
      matrix[i][j] = -1
    if matrix[i][j] == "A":
      matrix[i][j] = 0
    if matrix[i][j] == "W":
      matrix[i][j] = 10 
matrix

[[0, 4, 1, -1], [1, 1, 1, 10], [4, 1, 4, 1], [4, 1, 1, 1]]

In [ ]:
class Node:
  def __init__(self, i, j, h):
    self.i = i
    self.j = j
    self.h = h

In [ ]:
def makeNode(i, j, h):
  node = Node(i, j, h)
  return node

In [ ]:
matArr = []
for i in range(0, int(rules[1])):
  for j in range(0, int(rules[1])):
    matArr.append(makeNode(i, j, matrix[i][j]))

In [ ]:
temp = matArr
nodeArr = []
while temp != []:
    nodeArr.append(temp[:int(rules[1])])
    temp = temp[int(rules[1]):]

In [ ]:
def find_neighbours(arr):
  neighbors = []

  for i in range(len(arr)):
    for j, value in enumerate(arr[i]):
      if i == 0 or i == len(arr) - 1 or j == 0 or j == len(arr[i]) - 1:
        # corners
        new_neighbors = []
        if i != 0:
          new_neighbors.append({"top":arr[i - 1][j]})  # top neighbor
        if j != len(arr[i]) - 1:
          new_neighbors.append({"right":arr[i][j + 1]})  # right neighbor
        if i != len(arr) - 1:
          new_neighbors.append({"bottom":arr[i + 1][j]})  # bottom neighbor
        if j != 0:
          new_neighbors.append({"left":arr[i][j-1]})  # left neighbor

      else:
        # add neighbors
        new_neighbors = [
            {"top":arr[i - 1][j]},  # top neighbor
            {"right":arr[i][j + 1]},  # right neighbor
            {"bottom":arr[i + 1][j]},  # bottom neighbor
            {"left":arr[i][j - 1]}   # left neighbor
        ]

      neighbors.append({
          "value": value,
          "neighbors": new_neighbors})

  return neighbors

In [ ]:
neigh = find_neighbours(nodeArr)

In [ ]:
temp = neigh
new_neigh = []
while temp != []:
    new_neigh.append(temp[:int(rules[1])])
    temp = temp[int(rules[1]):]

In [ ]:
def getNeighbors(father):
  nei = father.get('neighbors')
  neighbors = []
  for item in nei:
    if 'top' in item:
      temp = item['top']
      temp.dir = 'U'
    if 'right' in item:
      temp = item['right']
      temp.dir = 'R'
    if 'bottom' in item:
      temp = item['bottom']
      temp.dir = 'D'
    if 'left' in item:
      temp = item['left']
      temp.dir = 'L'

  for item in nei:
    neighbors.append(list(item.values()))
  flat_list = [item for sublist in neighbors for item in sublist if item.h != 10]

  for item in flat_list:
    item.father = father.get('value')

  return flat_list

In [ ]:
def getPath(node):
  if (hasattr(node, 'father')) == False:
    print(node.h)
    return
  else:
    print(node.h)
    return getPath(node.father)

In [ ]:
def findMin(arra, close):
  newarr = []
  for item in arra:
    if item not in close:
      newarr.append(item) 
  newarr.sort(key=lambda x: x.h, reverse = True)
  if len(newarr) != 0: return newarr.pop()
  else: return None

In [ ]:
def greedyBestFirst(mat, neighs):
  path = ""
  cost = 1
  open = []
  close = []
  open.append(mat[0][0])
  while len(open) != 0:
    next = open.pop()
    close.append(next)
    cost+=next.h
    if next.h == -1:
      return(path + str(cost))
    neighbo = getNeighbors(neighs[next.i][next.j])
    obj = findMin(neighbo, close)
    if obj is None:
      close.pop()
      temp = close.pop()
      close=[]
      close.append(next)
      path+=temp.dir + "-"
      open.append(temp)
    elif obj not in open and obj not in close:
      path+= obj.dir + "-"
      open.append(obj)
  return("No Path")

In [ ]:
def addManhattan(matrix):
  indexI = 0
  indexJ = 0
  for i in range(len(matrix)):
    for j in range(len(matrix)):
      if matrix[i][j].h == -1:
        indexI = matrix[i][j].i
        indexJ = matrix[i][j].j
        
  for i in range(len(matrix)):
    for j in range(len(matrix)):
      sum = abs(indexI-i)+abs(indexJ-j)
      matrix[i][j].h+=sum

In [ ]:
def AStar(mat, neighs):
  addManhattan(mat)
  path = ""
  cost = 1
  open = []
  close = []
  open.append(mat[0][0])
  while len(open) != 0:
    next = open.pop()
    close.append(next)
    cost+=next.h
    if next.h == -1:
      return(path + str(cost))
    neighbo = getNeighbors(neighs[next.i][next.j])
    obj = findMin(neighbo, close)
    if obj is None:
      close.pop()
      temp = close.pop()
      close=[]
      close.append(next)
      path+=temp.dir + "-"
      open.append(temp)
    elif obj not in open and obj not in close:
      path+= obj.dir + "-"
      open.append(obj)
  return("No Path")

In [ ]:
if rules[0] == "A*":
  output = AStar(nodeArr, new_neigh)
  print(output)
  with open('output.txt', 'w') as f:
    f.write(output)

if rules[0] == "greedyBestFirst":
  output = greedyBestFirst(nodeArr, new_neigh)
  print(output)
  with open('output.txt', 'w') as f:
    f.write(output)

D-R-D-D-R-R-U-L-U-U-R-13
